In [1]:
%run 01_preprocess.ipynb

predictions_phoneme


/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


0.27.0
torgo_xlsr_finetune_F01 - Data Size: 14409
torgo_xlsr_finetune_F03 - Data Size: 8568
torgo_xlsr_finetune_F04 - Data Size: 9454
torgo_xlsr_finetune_M01 - Data Size: 6540
torgo_xlsr_finetune_M02 - Data Size: 9790
torgo_xlsr_finetune_M03 - Data Size: 9572
torgo_xlsr_finetune_M04 - Data Size: 11751
torgo_xlsr_finetune_M05 - Data Size: 8853
torgo_xlsr_finetune_F01_keep_all - Data Size: 16082
torgo_xlsr_finetune_F03_keep_all - Data Size: 16082
torgo_xlsr_finetune_F04_keep_all - Data Size: 16082
torgo_xlsr_finetune_M01_keep_all - Data Size: 16082
torgo_xlsr_finetune_M02_keep_all - Data Size: 16082
torgo_xlsr_finetune_M03_keep_all - Data Size: 16082
torgo_xlsr_finetune_M04__keep_all - Data Size: 16082
torgo_xlsr_finetune_M05_keep_all - Data Size: 16082


/tmp/ipykernel_21349/2226408003.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word_df.drop(columns=['word_count'], inplace=True)
/tmp/ipykernel_21349/2226408003.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentence_df.drop(columns=['word_count'], inplace=True)
/tmp/ipykernel_21349/2226408003.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word_df.drop(columns=['word_count'], inplace=True)
/tmp/ipykernel_21349/222640800

torgo_xlsr_finetune_F01_keep_all - Data Size: 12129
torgo_xlsr_finetune_F03_keep_all - Data Size: 12129
torgo_xlsr_finetune_F04_keep_all - Data Size: 12129
torgo_xlsr_finetune_M01_keep_all - Data Size: 12115
torgo_xlsr_finetune_M02_keep_all - Data Size: 12129
torgo_xlsr_finetune_M03_keep_all - Data Size: 12129
torgo_xlsr_finetune_M04__keep_all - Data Size: 12129
torgo_xlsr_finetune_M05_keep_all - Data Size: 12129

torgo_xlsr_finetune_F01_keep_all - Data Size: 3953
torgo_xlsr_finetune_F03_keep_all - Data Size: 3953
torgo_xlsr_finetune_F04_keep_all - Data Size: 3953
torgo_xlsr_finetune_M01_keep_all - Data Size: 3967
torgo_xlsr_finetune_M02_keep_all - Data Size: 3953
torgo_xlsr_finetune_M03_keep_all - Data Size: 3953
torgo_xlsr_finetune_M04__keep_all - Data Size: 3953
torgo_xlsr_finetune_M05_keep_all - Data Size: 3953

torgo_xlsr_finetune_F01 - Data Size: 10795
torgo_xlsr_finetune_F03 - Data Size: 6813
torgo_xlsr_finetune_F04 - Data Size: 7482
torgo_xlsr_finetune_M01 - Data Size: 4404
tor

In [2]:
notebook_login()

In [3]:
data_frame_current = dataframes_word_keep_all

In [4]:
MODEL_PATTERN="word_keep"

for df_name, df in dataframes_word_keep_all.items():
    print(f"{df_name} - Data Size: {len(df)}")

torgo_xlsr_finetune_F01_keep_all - Data Size: 336
torgo_xlsr_finetune_F03_keep_all - Data Size: 328
torgo_xlsr_finetune_F04_keep_all - Data Size: 296
torgo_xlsr_finetune_M01_keep_all - Data Size: 399
torgo_xlsr_finetune_M02_keep_all - Data Size: 534
torgo_xlsr_finetune_M03_keep_all - Data Size: 288
torgo_xlsr_finetune_M04__keep_all - Data Size: 449
torgo_xlsr_finetune_M05_keep_all - Data Size: 452


# some atypical speech pattern not usable in normal transducer only G2P EN have REPRENSENTATION

In [5]:
from g2p_en import G2p

g2p = G2p()

for df_name, df in data_frame_current.items():
    df['predictions_phoneme'] = df['predictions'].apply(lambda x: " ".join(g2p(x)))
    df['references_phoneme'] = df['references'].apply(lambda x: " ".join(g2p(x)))

    df.drop(['predictions', 'references'], axis=1, inplace=True)
# dataframes_word_keep_all

# Remove Duplicates

In [6]:
for df_name, df in data_frame_current.items():
    # Remove rows where columns are the same
    df = df[df['predictions_phoneme'] != df['references_phoneme']]
    
    # Remove rows where columns only differ by the number of spaces
    df = df[df.apply(lambda row: row['predictions_phoneme'].strip() != row['references_phoneme'].strip(), axis=1)]
    
    df.reset_index(drop=True, inplace=True)
# dataframes_word_keep_all

In [7]:
from sklearn.model_selection import train_test_split

ori_train_dataframes_word_keep_all = {}
test_dataframes_word_keep_all = {}

for df_name, df in dataframes_word_keep_all.items():
    # print(df)
    train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
    
    train_df_name = f"{df_name}"
    test_df_name = f"{df_name}_test"
    
    ori_train_dataframes_word_keep_all[train_df_name] = train_df
    test_dataframes_word_keep_all[test_df_name] = test_df

In [8]:
from sklearn.model_selection import train_test_split

train_dataframes_word_keep_all = {}
val_dataframes_word_keep_all = {} 

for df_name, df in ori_train_dataframes_word_keep_all.items():
    df = df.drop_duplicates(subset=['predictions_phoneme', 'references_phoneme'], keep=False)
    
    train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
    
    train_df_name = f"{df_name}_train"
    val_df_name = f"{df_name}_val"
    
    train_dataframes_word_keep_all[train_df_name] = train_df
    val_dataframes_word_keep_all[val_df_name] = val_df

In [9]:
train_dataframes_word_keep_all

{'torgo_xlsr_finetune_F01_keep_all_train':       predictions_phoneme references_phoneme
 15509   HH AY1 T R IH0 NG    HH IH1 T IH0 NG
 15236           S OY1 R T            S UW1 T
 15747             B AH1 D            B EH1 D
 15461            HH EY1 L           HH EH1 M
 14964             W EH1 T            M IY1 T
 ...                   ...                ...
 14905             V AE1 T            B AE1 T
 15637             W IY1 V            L EH1 R
 15671             T IH1 L            T EH1 L
 15914             B ER1 D            B ER1 D
 15812       F L IH1 K ER0      S L IH1 K ER0
 
 [255 rows x 2 columns],
 'torgo_xlsr_finetune_F03_keep_all_train':       predictions_phoneme references_phoneme
 15056         B R AE1 G T            B ER1 D
 14547               S AO1              S AY1
 14782            S IH1 NG      S EH1 V AH0 N
 14978             R EY1 T              EY1 T
 14476              SH ER1           SH EH1 R
 ...                   ...                ...
 15186         

In [10]:
for df_name, train_df in train_dataframes_word_keep_all.items():
    print(f"{df_name} - Train Set Size: {len(train_df)}")

for df_name, test_df in val_dataframes_word_keep_all.items():
    print(f"{df_name} - Val Set Size: {len(test_df)}")

for df_name, test_df in test_dataframes_word_keep_all.items():
    print(f"{df_name} - Test Set Size: {len(test_df)}")

torgo_xlsr_finetune_F01_keep_all_train - Train Set Size: 255
torgo_xlsr_finetune_F03_keep_all_train - Train Set Size: 252
torgo_xlsr_finetune_F04_keep_all_train - Train Set Size: 227
torgo_xlsr_finetune_M01_keep_all_train - Train Set Size: 300
torgo_xlsr_finetune_M02_keep_all_train - Train Set Size: 422
torgo_xlsr_finetune_M03_keep_all_train - Train Set Size: 225
torgo_xlsr_finetune_M04__keep_all_train - Train Set Size: 345
torgo_xlsr_finetune_M05_keep_all_train - Train Set Size: 345
torgo_xlsr_finetune_F01_keep_all_val - Val Set Size: 29
torgo_xlsr_finetune_F03_keep_all_val - Val Set Size: 28
torgo_xlsr_finetune_F04_keep_all_val - Val Set Size: 26
torgo_xlsr_finetune_M01_keep_all_val - Val Set Size: 34
torgo_xlsr_finetune_M02_keep_all_val - Val Set Size: 47
torgo_xlsr_finetune_M03_keep_all_val - Val Set Size: 26
torgo_xlsr_finetune_M04__keep_all_val - Val Set Size: 39
torgo_xlsr_finetune_M05_keep_all_val - Val Set Size: 39
torgo_xlsr_finetune_F01_keep_all_test - Test Set Size: 34
torg

In [11]:
from jiwer import wer,cer
# Extract reference and hypothesis sentences

for df_name, df in test_dataframes_word_keep_all.items():
    # Assuming your DataFrame has 'references_phoneme' and 'predictions_phoneme' columns
    references = df['references_phoneme'].tolist()
    hypotheses = df['predictions_phoneme'].tolist()
    
    # Calculate WER and CER for the current speaker
    wer_score = wer(references, hypotheses)
    cer_score = cer(references, hypotheses)
    
    print(f'Speaker: {df_name}')
    print(f'Word Error Rate (WER): {wer_score:.2%}')
    print(f'Character Error Rate (CER): {cer_score:.2%}')

Speaker: torgo_xlsr_finetune_F01_keep_all_test
Word Error Rate (WER): 57.63%
Character Error Rate (CER): 42.31%
Speaker: torgo_xlsr_finetune_F03_keep_all_test
Word Error Rate (WER): 50.83%
Character Error Rate (CER): 36.64%
Speaker: torgo_xlsr_finetune_F04_keep_all_test
Word Error Rate (WER): 60.19%
Character Error Rate (CER): 41.63%
Speaker: torgo_xlsr_finetune_M01_keep_all_test
Word Error Rate (WER): 68.99%
Character Error Rate (CER): 57.64%
Speaker: torgo_xlsr_finetune_M02_keep_all_test
Word Error Rate (WER): 65.78%
Character Error Rate (CER): 44.94%
Speaker: torgo_xlsr_finetune_M03_keep_all_test
Word Error Rate (WER): 72.22%
Character Error Rate (CER): 47.69%
Speaker: torgo_xlsr_finetune_M04__keep_all_test
Word Error Rate (WER): 74.84%
Character Error Rate (CER): 51.27%
Speaker: torgo_xlsr_finetune_M05_keep_all_test
Word Error Rate (WER): 64.78%
Character Error Rate (CER): 49.09%


# toCSV

In [12]:
import os

output_folder = f'data/{MODEL_PATTERN}'

os.makedirs(output_folder, exist_ok=True)

for df_name, train_df in train_dataframes_word_keep_all.items():
    train_df.to_csv(os.path.join(output_folder, f'{df_name}.csv'), index=False)

for df_name, val_df in val_dataframes_word_keep_all.items():
    val_df.to_csv(os.path.join(output_folder, f'{df_name}.csv'), index=False)

for df_name, test_df in test_dataframes_word_keep_all.items():
    test_df.to_csv(os.path.join(output_folder, f'{df_name}.csv'), index=False)

# to json

In [13]:
def convert_csv_to_json(csv_path, json_output_path):
    df = pd.read_csv(csv_path)
    json_data = df.to_json(orient='records')

    with open(json_output_path, 'w') as json_file:
        json_file.write(json_data)

csv_folder = f'data/{MODEL_PATTERN}'
json_folder = f'data/{MODEL_PATTERN}'

os.makedirs(json_folder, exist_ok=True)

for csv_file_name in os.listdir(csv_folder):
    if csv_file_name.endswith('.csv'):
        csv_file_path = os.path.join(csv_folder, csv_file_name)
        json_file_name = csv_file_name.replace('.csv', '.json')
        json_output_file_path = os.path.join(json_folder, json_file_name)

        convert_csv_to_json(csv_file_path, json_output_file_path)

In [14]:
import os
from datasets import load_dataset

loaded_datasets = {}

for speaker in SPEAKERS:
    file_pattern = f'torgo_xlsr_finetune_{speaker}_{{split}}.json'

    loaded_datasets[speaker] = load_dataset('json', data_files={
        'train': os.path.join(f'data/{MODEL_PATTERN}', file_pattern.format(split='train')),
        'valid': os.path.join(f'data/{MODEL_PATTERN}', file_pattern.format(split='val')),
        'test': os.path.join(f'data/{MODEL_PATTERN}', file_pattern.format(split='test')),
    })

for speaker, dataset in loaded_datasets.items():
    print(f"SPEAKER {speaker} - Train Set Size: {len(dataset['train'])}")
    print(f"SPEAKER {speaker} - Valid Set Size: {len(dataset['valid'])}")
    print(f"SPEAKER {speaker} - Test Set Size: {len(dataset['test'])}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

...

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

SPEAKER F01 - Train Set Size: 233
SPEAKER F01 - Valid Set Size: 26
SPEAKER F01 - Test Set Size: 31
SPEAKER F03 - Train Set Size: 267
SPEAKER F03 - Valid Set Size: 30
SPEAKER F03 - Test Set Size: 36
SPEAKER F04 - Train Set Size: 254
SPEAKER F04 - Valid Set Size: 29
SPEAKER F04 - Test Set Size: 34
SPEAKER M01 - Train Set Size: 271
SPEAKER M01 - Valid Set Size: 31
SPEAKER M01 - Test Set Size: 36
SPEAKER M02 - Train Set Size: 317
SPEAKER M02 - Valid Set Size: 36
SPEAKER M02 - Test Set Size: 41
SPEAKER M03 - Train Set Size: 213
SPEAKER M03 - Valid Set Size: 24
SPEAKER M03 - Test Set Size: 29
SPEAKER M04 - Train Set Size: 336
SPEAKER M04 - Valid Set Size: 38
SPEAKER M04 - Test Set Size: 44
SPEAKER M05 - Train Set Size: 313
SPEAKER M05 - Valid Set Size: 35
SPEAKER M05 - Test Set Size: 41


In [15]:
loaded_datasets['F01']

DatasetDict({
    train: Dataset({
        features: ['predictions_phoneme', 'references_phoneme'],
        num_rows: 233
    })
    valid: Dataset({
        features: ['predictions_phoneme', 'references_phoneme'],
        num_rows: 26
    })
    test: Dataset({
        features: ['predictions_phoneme', 'references_phoneme'],
        num_rows: 31
    })
})

In [16]:
# Initialize the tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

In [17]:
def train_model_for_speaker(speaker_id):
    print(speaker_id)
    # Apply preprocess_function to train_data and val_data
    train_data=loaded_datasets[speaker_id]['train']
    val_data=loaded_datasets[speaker_id]['valid']
    train_data = train_data.map(preprocess_function, batched=True)
    val_data = val_data.map(preprocess_function, batched=True)

    # Prepare DataLoader for training and validation
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=16)

    output_dir = f"torgo_spell_correction_{MODEL_PATTERN}_{speaker_id}"

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=40,
        predict_with_generate=True,
        push_to_hub=True,
        logging_steps=100
    )

    print("Train Dataset:", train_data)
    print("Validation Dataset:", val_data)
    print("Tokenizer:", tokenizer)
    print("Training Arguments:", training_args)

    model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # The Seq2SeqTrainer is created with the defined model, training arguments, datasets, tokenizer, and data_collator
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=val_data,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    trainer.train()



In [ ]:
train_model_for_speaker('F01')
train_model_for_speaker('F03')
train_model_for_speaker('F04')
train_model_for_speaker('M01')
train_model_for_speaker('M02')
train_model_for_speaker('M03')
train_model_for_speaker('M04')
train_model_for_speaker('M05')

F01


Map:   0%|          | 0/233 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Train Dataset: Dataset({
    features: ['predictions_phoneme', 'references_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 233
})
Validation Dataset: Dataset({
    features: ['predictions_phoneme', 'references_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 26
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


...

Map:   0%|          | 0/267 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Train Dataset: Dataset({
    features: ['predictions_phoneme', 'references_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 267
})
Validation Dataset: Dataset({
    features: ['predictions_phoneme', 'references_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 30
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,No log,6.641427
2,No log,5.393003
3,No log,3.776828
4,No log,2.973657


In [ ]:
from transformers import BartForConditionalGeneration

SPEAKERS = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']
models = {}

for speaker in SPEAKERS:
    model_name = f"matrixcc/torgo_spell_correction_{MODEL_PATTERN}_{speaker_id}"
    models[speaker] = BartForConditionalGeneration.from_pretrained(model_name)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for speaker, model in models.items():
    models[speaker] = model.to(device)
    models[speaker].eval()

In [ ]:
def evaluate_speaker(model, tokenizer, speaker_id):
    predictions = []
    references = []
    test_dataset = loaded_datasets[speaker_id]['test']
    test_dataset = test_dataset.map(preprocess_function, batched=True)
    model.eval()

    for example in test_dataset:
        input_text = f"{example['predictions_phoneme']}"
        with torch.no_grad():
            input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
            outputs = model.generate(input_ids=input_ids, max_length=max_length)

        predicted_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

        references.append(example['references_phoneme'])
        predictions.append(predicted_sentence)

    # Verify that the number of predictions and references are the same
    if len(predictions) == len(references):
        print("Number of predictions and references are the same.")
    else:
        print("Mismatch in the number of predictions and references.")

    # Print the number of predictions and references
    print("Number of predictions:", len(predictions))
    print("Number of references:", len(references))
    # print the length of the dataset
    print("Number of rows in dataset:", len(test_dataset))

    # Assuming 'predictions' and 'references' are your sequences
    # Calculate Word Error Rate (WER)
    wer_value = wer(predictions, references)
    wer_percentage = wer_value * 100
    print(f"WER for {speaker_id}: {wer_percentage:.2f}%")

    # Calculate Character Error Rate (CER)
    cer_value = cer(predictions, references)
    cer_percentage = cer_value * 100
    print(f"CER for {speaker_id}: {cer_percentage:.2f}%")

In [ ]:
for speaker_id, model in models.items():
    evaluate_speaker(model, tokenizer, speaker_id)